# Two Bidding Zone with Transmission

Single bidding zone optimisation problem is expanded to
\begin{equation}
  \min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}
\end{equation}
such that
\begin{align}
  g_{i,s} &\leq G_{i,s} \\
  g_{i,s} &\geq 0 \\
  \sum_s g_{i,s} - \sum_\ell K_{i\ell} f_\ell &= d_i & \text{KCL} \\
  |f_\ell| &\leq F_\ell & \text{line limits}  \\
  \sum_\ell C_{\ell c} x_\ell f_\ell &= 0 & \text{KVL} 
\end{align}

In this example, we connect the previous South African electricity system with a hydro generation unit in Mozambique through a single transmission line. Note that because a single transmission line will not result in any cycles, we can neglect KVL in this case.

We are given the following data (all in MW):

In [1]:
import pyomo.environ as pe
import pandas as pd

In [2]:
capacities = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {"Hydro": 1200},
}

In [3]:
transmission = 500

In [4]:
loads = {"South Africa": 42000, "Mozambique": 650}

In [5]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
    "Hydro": 0,
}

In [6]:
m = pe.ConcreteModel()
m.dual = pe.Suffix(direction = pe.Suffix.IMPORT)

In [7]:
valid_pairs = [(c, s) for c in capacities for s in capacities[c]]

m.country_tech_pairs = pe.Set(initialize=valid_pairs, dimen=2)

In [8]:
m.country_tech_pairs.pprint()

country_tech_pairs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    5 : {('South Africa', 'Coal'), ('South Africa', 'Wind'), ('South Africa', 'Gas'), ('South Africa', 'Oil'), ('Mozambique', 'Hydro')}


In [9]:
# Define variable only for existing pairs
m.g = pe.Var(m.country_tech_pairs, within=pe.NonNegativeReals)

In [10]:
m.f = pe.Var()

In [11]:
m.cost = pe.Objective(
    expr=sum(marginal_costs[s] * m.g[c, s] for (c, s) in m.country_tech_pairs)
)

In [12]:
@m.Constraint(m.country_tech_pairs)
def generator_limits (m,c,s):
    return m.g[c,s] <= capacities[c].get(s,0)

In [13]:
m.generator_limits.pprint()

generator_limits : Size=5, Index=country_tech_pairs, Active=True
    Key                      : Lower : Body                 : Upper   : Active
     ('Mozambique', 'Hydro') :  -Inf :  g[Mozambique,Hydro] :  1200.0 :   True
    ('South Africa', 'Coal') :  -Inf : g[South Africa,Coal] : 35000.0 :   True
     ('South Africa', 'Gas') :  -Inf :  g[South Africa,Gas] :  8000.0 :   True
     ('South Africa', 'Oil') :  -Inf :  g[South Africa,Oil] :  2000.0 :   True
    ('South Africa', 'Wind') :  -Inf : g[South Africa,Wind] :  3000.0 :   True


In [14]:
@m.Constraint(capacities.keys())
def kcl(m, c):
    sign = -1 if c == "Mozambique" else 1
    return sum(m.g[c, s] for s in capacities[c]) - sign * m.f == loads[c]

In [15]:
m.kcl.pprint()

kcl : Size=2, Index={Mozambique, South Africa}, Active=True
    Key          : Lower   : Body                                                                                        : Upper   : Active
      Mozambique :   650.0 :                                                                     g[Mozambique,Hydro] + f :   650.0 :   True
    South Africa : 42000.0 : g[South Africa,Coal] + g[South Africa,Wind] + g[South Africa,Gas] + g[South Africa,Oil] - f : 42000.0 :   True


In [16]:
m.line_limit = pe.Constraint(expr = (-transmission, m.f, transmission))
m.line_limit.pprint()

line_limit : Size=1, Index=None, Active=True
    Key  : Lower  : Body : Upper : Active
    None : -500.0 :    f : 500.0 :   True


In [17]:
pe.SolverFactory("gurobi").solve(m).write() # Can be solved wwith "appsi-highs"

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 1260000.0
  Upper bound: 1260000.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 6
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 6
  Number of nonzeros: 14
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (s

In [18]:
m.cost()

1260000.0

In [19]:
m.f()

-500.0

In [20]:
pd.Series(m.g.get_values()).unstack()

,Coal,Gas,Hydro,Oil,Wind
Mozambique,NaN,NaN,1150.0,NaN,NaN
South Africa,35000.0,3500.0,NaN,0.0,3000.0


In [21]:
pd.Series(m.dual.values(), m.dual.keys())

generator_limits[South Africa,Coal]   -30.0
generator_limits[South Africa,Wind]   -60.0
generator_limits[South Africa,Gas]      0.0
generator_limits[South Africa,Oil]      0.0
generator_limits[Mozambique,Hydro]      0.0
kcl[Mozambique]                         0.0
kcl[South Africa]                      60.0
[None]                                 60.0
dtype: float64

# Deactivate line constraints, and fixed one variables to a pre-defined value

In [22]:
m.line_limit.deactivate()
m.g["South Africa", "Coal"].fix(34000)

In [23]:
pe.SolverFactory("gurobi").solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 1287000.0
  Upper bound: 1287000.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 6
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 6
  Number of nonzeros: 10
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (s

In [24]:
m.cost()

1287000.0

In [25]:
m.f()

-550.0

In [26]:
pd.Series(m.g.get_values()).unstack()

,Coal,Gas,Hydro,Oil,Wind
Mozambique,NaN,NaN,1200.0,NaN,NaN
South Africa,34000.0,4450.0,NaN,0.0,3000.0
